In [ ]:
import os

# this get our current location in the file system
import inspect
HERE_PATH = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))

In [ ]:
from pprint import pprint
import numpy as np
import random
import time

import tools

from discrete import DiscreteLearner
from discrete import DiscretePlayer

In [ ]:
class DiscreteRunner(object):

    def __init__(self, n_hypothesis, player_symbols, known_symbols={}, target_index=None):

        self.n_hypothesis = n_hypothesis
        self.learner = DiscreteLearner(n_hypothesis, known_symbols)
        self.player = DiscretePlayer(n_hypothesis, player_symbols, target_index)

    def reset_learner(self, updated_known_symbols):
        self.learner = DiscreteLearner(self.n_hypothesis, updated_known_symbols)

    def update_target_index(self, new_target_index):
        self.player.update_target_index(new_target_index)

    def step(self, planning_method):
        flash_pattern = self.learner.get_next_flash_pattern(planning_method)
        feedback_symbol = self.player.get_feedback_symbol(flash_pattern)
        self.learner.update(flash_pattern, feedback_symbol)

    def run_until_solved(self, planning_method, n_abort_steps=np.inf):

        aborted = False
        inconsistent = False

        n_steps = 0
        step_durations = []
        while not self.learner.is_solved():
            start_time = time.time()
            n_steps+=1

            self.step(planning_method)

            if self.learner.is_inconsistent():
                aborted = True
                inconsistent = True
                break

            if n_steps == n_abort_steps:
                aborted = True
                break

            step_durations.append(time.time() - start_time)

        # logging
        run_info = {}
        # log basic data
        run_info['aborted'] = aborted
        run_info['inconsistent'] = inconsistent
        run_info['n_steps'] = n_steps
        run_info['avg_steps_duration'] = np.mean(step_durations)
        run_info['player_target_index'] = self.player.target_index
        # log history
        run_info['learner_known_symbols'] = self.learner.known_symbols.copy(),
        run_info['hypothesis_validity_history'] = self.learner.hypothesis_validity_history.copy(),
        run_info['flash_history'] = self.learner.flash_history.copy(),
        run_info['symbol_history'] = self.learner.symbol_history.copy(),
        run_info['hypothesis_labels'] = self.learner.hypothesis_labels.copy()
        # if solved log what was understood
        if not aborted:
            learner_solution_index = self.learner.get_solution_index()
            learner_interpretation = self.learner.compute_symbols_belief_for_hypothesis(learner_solution_index)

            run_info['learner_solution_index'] = learner_solution_index
            run_info['learner_interpretation'] = learner_interpretation

        return run_info

In [ ]:
N_ABORT_STEPS = np.inf
TARGET_INDEX = None
PLANNING_METHOD = 'even_uncertainty'


def run(seed, code_length, n_hypothesis, n_button_per_label, are_labels_known):

    tools.set_seed(seed, verbose=False)

    player_symbols = {
        True: [str(i) for i in range(n_button_per_label)],
        False: [str(i) for i in range(n_button_per_label, 2*n_button_per_label)]
    }

    known_symbols = {}
    if are_labels_known:
        for label in [True, False]:
            for symbol in player_symbols[label]:
                known_symbols[symbol] = label

    runner = DiscreteRunner(n_hypothesis, player_symbols, known_symbols, TARGET_INDEX)

    result = {
        'steps_per_solution' : [],
        'avg_steps_duration_per_solution': []
    }
    for _ in range(code_length):
        run_info = runner.run_until_solved(PLANNING_METHOD, N_ABORT_STEPS)
        runner.reset_learner(run_info['learner_interpretation'])
        runner.update_target_index(TARGET_INDEX)

        # print('Identified {} from {} in {} steps.'.format(run_info['learner_solution_index'], run_info['player_target_index'], run_info['n_steps']))

        result['steps_per_solution'].append(run_info['n_steps'])
        result['avg_steps_duration_per_solution'].append(run_info['avg_steps_duration'])

    return result

In [ ]:
SEED = np.random.randint(0, 10000)
if SEED is not None:
    tools.set_seed(SEED, verbose=False)

In [ ]:
import math
import itertools
from tinydb import TinyDB, Query

DB_FILENAME = os.path.join(HERE_PATH, 'results.json')
db = TinyDB(DB_FILENAME)
user = Query()

N_EXPERIMENTS = 100
XP_PARAMS = {
    'seed': range(N_EXPERIMENTS),
    'code_length': [1, 2, 3, 4, 5],
    'n_hypothesis': [4, 6, 8, 10, 12],
    'n_button_per_label': [1, 2, 3, 4, 5],
    'are_labels_known': [True, False]
}

In [ ]:
param_keys, param_values = zip(*XP_PARAMS.items())

n_experiments = sum(1 for _ in itertools.product(*param_values))
digits = int(math.log10(n_experiments))+1

for i_exp, param_combination in enumerate(itertools.product(*param_values)):

    i_exp_corrected = i_exp + 1
    i_str = f"{i_exp_corrected:{digits}}"
    n_exp_str = f"{n_experiments:{digits}}"

    print('{}:{} '.format(i_str , n_exp_str), end='\b')

    xp = {}
    xp['params'] = dict(zip(param_keys, param_combination))

    if db.search(user.params == xp['params']):
        print(' - Skipping')
        continue
    else:
        print(' - Running')
        xp['results'] = run(**xp['params'])
        db.insert(xp)